In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import RobustScaler, Normalizer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit, StratifiedKFold, TunedThresholdClassifierCV

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report

import sklearn

import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors

from sklearn.metrics import make_scorer, recall_score, precision_score, f1_score, accuracy_score

from sklearn import svm
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import cross_validate
from sklearn.metrics import auc
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer , make_column_transformer
from sklearn.impute import SimpleImputer, KNNImputer

### Область работы 2 (выполнение лучшей модели)

In [3]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [4]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [5]:
df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)

In [6]:
df.drop_duplicates(inplace=True)

In [14]:
X_train = df.drop(['species'], axis = 1)
y_train = df['species']

In [15]:
# Блок обучения модели

In [16]:
transformer_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent', add_indicator=False)),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

transformer_num = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=False)),
    ('scaler', StandardScaler())
])

In [17]:
cat_features = ['island','sex']
num_features = ['bill_depth_mm', 'bill_length_mm', 'body_mass_g', 'flipper_length_mm']

In [18]:
CT = ColumnTransformer([
        ("num", transformer_num, num_features),
        ("cat", transformer_cat, cat_features),
        ]).set_output(transform="pandas")

display(CT)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['bill_depth_mm', 'bill_length_mm',
                                  'body_mass_g', 'flipper_length_mm']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['island', 'sex'])])

In [19]:
model = Pipeline([
    ('preprocessing',CT),
    ('classifier', svm.SVC(C=0.1, gamma = 'scale'))
])
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['bill_depth_mm',
                                                   'bill_length_mm',
                                                   'body_mass_g',
                                                   'flipper_length_mm']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['island', 'sex'])])),
                ('classifier', SVC(C=0.1))])

In [7]:
# Блок предсказания с использованием тестового набора

In [20]:
y_pred = model.predict(X_test)

In [21]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict = y_pred